In [ ]:
# STEP 1: Upload the Excel file into Colab
from google.colab import files
uploaded = files.upload()   # Choose Airbnb_Open_Data.xlsx

import os
print("Uploaded files:", os.listdir())

In [ ]:
# STEP 2: Load the dataset
import pandas as pd
import numpy as np

INPUT_PATH = "Airbnb_Open_Data.xlsx"   # name after upload
SHEET_NAME = "in"

df = pd.read_excel(INPUT_PATH, sheet_name=SHEET_NAME)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()

In [ ]:
# STEP 3: Data cleaning & preprocessing

# Pick features (avoid high-cardinality 'neighbourhood')
features = [
    'room type', 'neighbourhood group',
    'lat', 'long',
    'accommodates' if 'accommodates' in df.columns else None,
    'bathrooms' if 'bathrooms' in df.columns else None,
    'bedrooms' if 'bedrooms' in df.columns else None,
    'beds' if 'beds' in df.columns else None,
    'minimum nights' if 'minimum nights' in df.columns else None,
    'number of reviews' if 'number of reviews' in df.columns else None,
    'availability 365' if 'availability 365' in df.columns else None
]
features = [f for f in features if f is not None]

TARGET = "price"
data = df[features + [TARGET]].copy()

# Remove missing/invalid prices
data = data.dropna(subset=[TARGET])
data = data[(data[TARGET] > 0) & (data[TARGET] <= 5000)]

# Convert to numeric where possible
for col in ['bathrooms','bedrooms','beds','accommodates','minimum nights','number of reviews','availability 365']:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors="coerce")

# Fill missing
numeric_features = [c for c in features if pd.api.types.is_numeric_dtype(data[c])]
categorical_features = [c for c in features if c not in numeric_features]
data[numeric_features] = data[numeric_features].fillna(data[numeric_features].median())
data[categorical_features] = data[categorical_features].fillna("missing")

# Sample for speed (optional)
if len(data) > 5000:
    data = data.sample(5000, random_state=42)

# Log-transform target
data["log_price"] = np.log1p(data[TARGET])

print("Final dataset shape:", data.shape)
data.head()

In [ ]:
# STEP 4: Train models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

X = data[features]
y = data["log_price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Preprocessing
numeric_transformer = Pipeline([("scaler", StandardScaler())])
categorical_transformer = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# Models
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
}

results = {}
for name, model in models.items():
    pipe = Pipeline([("pre", preprocessor), ("model", model)])
    pipe.fit(X_train, y_train)

    preds_log = pipe.predict(X_test)
    preds_price = np.expm1(preds_log)
    actual_price = np.expm1(y_test)

    rmse = mean_squared_error(actual_price, preds_price, squared=False)
    mae = mean_absolute_error(actual_price, preds_price)
    r2 = r2_score(actual_price, preds_price)

    results[name] = {"rmse": rmse, "mae": mae, "r2": r2, "pipeline": pipe}

print("Results:")
for k,v in results.items():
    print(f"{k}: RMSE={v['rmse']:.2f}, MAE={v['mae']:.2f}, R2={v['r2']:.3f}")

In [ ]:
# STEP 5: Plot predictions vs actual
import matplotlib.pyplot as plt

best_model = min(results, key=lambda k: results[k]["rmse"])
pipe = results[best_model]["pipeline"]

preds = np.expm1(pipe.predict(X_test))
actual = np.expm1(y_test)

plt.figure(figsize=(7,6))
plt.scatter(actual, preds, alpha=0.3)
plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], color="red")
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title(f"Actual vs Predicted Price ({best_model})")
plt.show()

In [ ]:
# STEP 6: Save the best model
import pickle

with open("best_airbnb_model.pkl", "wb") as f:
    pickle.dump(results[best_model]["pipeline"], f)

print("Saved best model as best_airbnb_model.pkl")

In [ ]:
# Example: Load saved model & make prediction
with open("best_airbnb_model.pkl", "rb") as f:
    model = pickle.load(f)

example = pd.DataFrame([{
    'room type': 'Private room',
    'neighbourhood group': 'Brooklyn',
    'lat': 40.7,
    'long': -73.9,
    'accommodates': 2,
    'bathrooms': 1,
    'bedrooms': 1,
    'beds': 1,
    'minimum nights': 3,
    'number of reviews': 20,
    'availability 365': 180
}])

pred_price = np.expm1(model.predict(example)[0])
print("Predicted price:", round(pred_price,2))